# 🤗 `transformers`

More info about using `transformers` models with outlines [here](https://dottxt-ai.github.io/outlines/latest/reference/models/transformers/)

Table of Contents:
- [JSON Generation](#json-generation)
- [Text Generation](#text-generation)


## JSON Generation

Example based on https://dottxt-ai.github.io/outlines/latest/examples/extraction/

In [1]:
from enum import Enum

import jinja2
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from pydantic import BaseModel

from outlines_haystack.generators.transformers import TransformersJSONGenerator

In [2]:
import torch

device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"

In [3]:
class Pizza(str, Enum):
    margherita = "Margherita"
    calzone = "Calzone"


class Order(BaseModel):
    pizza: Pizza
    number: int

In [4]:
prompt_template = """You are the owner of a pizza parlor. Customers
send you orders from which you need to extract:

1. The pizza that is ordered
2. The number of pizzas

# EXAMPLE

ORDER: I would like one Margherita pizza
RESULT: {"pizza": "Margherita", "number": 1}

# OUTPUT INSTRUCTIONS

Answer in valid JSON. Here are the different objects relevant for the output:

Order:
    pizza (str): name of the pizza
    number (int): number of pizzas

Return a valid JSON of type "Order"

# OUTPUT

ORDER: {{ order }}
RESULT: """

In [7]:
generator = TransformersJSONGenerator(model_name="microsoft/Phi-3-mini-4k-instruct", schema_object=Order, device=device)

### Standalone

In [8]:
generator.warm_up()

Loading weights:   0%|          | 0/195 [00:00<?, ?it/s]

In [12]:
# we use Jinja2 to render the template
prompt = jinja2.Template(prompt_template).render(order="Is it possible to have 12 margheritas?")
generator.run(prompt=prompt, generation_kwargs={"temperature": 0.5, "do_sample": True})

{'structured_replies': ['{" pizza": " Margherita", "number": 1 1 1 1']}

### In a Pipeline

In [13]:
pipeline = Pipeline()
pipeline.add_component(
    instance=PromptBuilder(template=prompt_template, required_variables=["order"]), name="prompt_builder"
)
pipeline.add_component(
    instance=TransformersJSONGenerator(
        model_name="microsoft/Phi-3-mini-4k-instruct",
        schema_object=Order,
        device=device,
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: TransformersJSONGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [14]:
pipeline.run(
    {
        "prompt_builder": {"order": "Is it possible to have 12 margheritas?"},
        "llm": {"generation_kwargs": {"temperature": 0.5, "do_sample": True}},
    }
)

Loading weights:   0%|          | 0/195 [00:00<?, ?it/s]

{'llm': {'structured_replies': ['{"pizza": "Margherit  a", "number": 12 1 ']}}

## Choice Generation

In [15]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.transformers import TransformersChoiceGenerator

In [16]:
generator = TransformersChoiceGenerator(
    model_name="microsoft/Phi-3-mini-4k-instruct",
    choices=["Positive", "Negative"],
    device=device,
)

### Standalone

In [17]:
generator.warm_up()

Loading weights:   0%|          | 0/195 [00:00<?, ?it/s]

In [ ]:
generator.run(
    prompt="Classify the following statement: 'I love pizza'", generation_kwargs={"temperature": 0.5, "do_sample": True}
)

{'choice': ' Positive '}

### In a Pipeline

In [22]:
prompt_template = "Classify the following statement: '{{statement}}'"

pipeline = Pipeline()
pipeline.add_component(
    instance=PromptBuilder(template=prompt_template, required_variables=["statement"]), name="prompt_builder"
)
pipeline.add_component(
    instance=TransformersChoiceGenerator(
        model_name="microsoft/Phi-3-mini-4k-instruct",
        choices=["Positive", "Negative"],
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: TransformersChoiceGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [23]:
pipeline.run(
    {
        "prompt_builder": {"statement": "I love Italian food"},
        "llm": {"generation_kwargs": {"temperature": 0.5, "do_sample": True}},
    }
)

Loading weights:   0%|          | 0/195 [00:00<?, ?it/s]

{'llm': {'choice': ' P  o  s  i  ti  ve  '}}

## Text Generation

In [24]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.transformers import TransformersTextGenerator

In [25]:
generator = TransformersTextGenerator(
    model_name="microsoft/Phi-3-mini-4k-instruct",
    device=device,
)

### Standalone

In [26]:
generator.warm_up()

Loading weights:   0%|          | 0/195 [00:00<?, ?it/s]

In [27]:
generator.run(prompt="What is the capital of Italy?")

{'replies': ['\n\n# Answer\nThe capital of Italy is Rome.']}

### In a Pipeline

In [28]:
prompt_template = "What is the capital of {{country}}?"

pipeline = Pipeline()
pipeline.add_component(
    instance=PromptBuilder(template=prompt_template, required_variables=["country"]), name="prompt_builder"
)
pipeline.add_component(
    instance=TransformersTextGenerator(
        model_name="microsoft/Phi-3-mini-4k-instruct",
        device=device,
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: TransformersTextGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [29]:
pipeline.run({"prompt_builder": {"country": "France"}})

Loading weights:   0%|          | 0/195 [00:00<?, ?it/s]

{'llm': {'replies': ['\n\n# Answer\nThe capital of France is Paris.']}}